In [1]:
import csv
import os
import numpy as np
from datetime import datetime
from stable_baselines3 import PPO

In [5]:
def predict_and_store_rl(time_str, model_path="ppo_vm_batch5.zip", output_file="predictions.csv"):
 

    try:
        time_obj = datetime.strptime(time_str, "%H:%M:%S")
    except ValueError:
        raise ValueError("Time must be in HH:MM:SS format (e.g., '12:34:56')")
    
    # Extract features from time (normalize to [0, 1])
    hour = time_obj.hour / 23.0
    minute = time_obj.minute / 59.0
    
    # Create observation with shape (100, 2)
    # For simplicity, repeat the [hour, minute] vector 100 times
    # Replace this with your model's expected sequence if different (e.g., time series)
    observation = np.array([[hour, minute]] * 100, dtype=np.float32)
    assert observation.shape == (100, 2), f"Observation shape {observation.shape} does not match expected (100, 2)"
    
    # Load the trained PPO model
    try:
        model = PPO.load(model_path)
    except Exception as e:
        raise ValueError(f"Failed to load PPO model from {model_path}: {str(e)}")
    
    # Predict using the PPO model
    try:
        action, _ = model.predict(observation, deterministic=True)
    except Exception as e:
        raise ValueError(f"Prediction failed: {str(e)}")
    
    # Action is [predicted_total_cpu, predicted_total_mem, predicted_total_net]
    predicted_cpu, predicted_mem, predicted_net = action
    
    # Ensure predictions are within reasonable bounds
    predicted_cpu = max(0, min(predicted_cpu, 5000))  # CPU in MHz
    predicted_mem = max(0, min(predicted_mem, 10000))  # Memory in KB
    predicted_net = max(0, min(predicted_net, 2000))   # Network in KB/s
    
    # Round predictions to 2 decimal places
    predicted_cpu = round(predicted_cpu, 2)
    predicted_mem = round(predicted_mem, 2)
    predicted_net = round(predicted_net, 2)
    
    # Calculate VMs needed (e.g., 500 MHz per VM)
    vms_needed = int(np.ceil(predicted_cpu / 500))
    
    # Prepare data to store
    prediction_data = {
        "Time": time_str,
        "CPU_usage_MHZ": predicted_cpu,
        "Memory_usage_KB": predicted_mem,
        "Network_received_throughput_KB_s": predicted_net,
        "VMs_needed": vms_needed
    }
    
    # Check if file exists to determine if headers are needed
    file_exists = os.path.exists(output_file)
    
    # Store predictions in CSV
    with open(output_file, mode='a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=prediction_data.keys())
        
        # Write header if file is new
        if not file_exists:
            writer.writeheader()
        
        # Write prediction data
        writer.writerow(prediction_data)
    
    print(f"Stored predictions for {time_str} in {output_file}")
    return prediction_data

In [6]:
# Test with a sample time
sample_time = "12:34:56"
result = predict_and_store_rl(sample_time, model_path="ppo_vm_batch5.zip")
print("Predictions:", result)

ValueError: not enough values to unpack (expected 3, got 1)

In [ ]:
num_vms = max(vms_cpu, vms_mem, vms_net)+1
print(f"VMs needed: {num_vms}")

In [ ]:
input_csv = "predictions.csv"
df = pd.read_csv(input_csv)

# Define VM capacities
VM_CPU_CAPACITY_MHZ = 2000
VM_MEM_CAPACITY_KB = 4096
VM_NET_CAPACITY_KB_s = 1024

# Calculate VMs needed for each timestamp
def calculate_vms(row):
    vms_cpu = math.ceil(row['CPU_usage_MHZ'] / VM_CPU_CAPACITY_MHZ)
    vms_mem = math.ceil(row['Memory_usage_KB'] / VM_MEM_CAPACITY_KB)
    vms_net = math.ceil(row['Network_received_throughput_KB_s'] / VM_NET_CAPACITY_KB_s)
    return max(vms_cpu, vms_mem, vms_net)

# Add VMs_needed column
df['VMs_needed'] = df.apply(calculate_vms, axis=1)

# Save to new CSV with VM count
output_csv = "vm_allocations.csv"
df.to_csv(output_csv, index=False)
print(f"Saved VM allocations to {output_csv}")